In [1]:
from datetime import datetime
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import pandas as pd
import pickle
from sklearn.metrics import root_mean_squared_error

In [2]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [3]:
client.create_experiment(name="my-cool-experiment")

'2'

In [4]:
client.search_experiments()

[<Experiment: artifact_location='file:///d:/rebeka/developer/courses/machine-learning-ops-zoomcamp/code/2_mlflow/mlruns/2', creation_time=1716986037626, experiment_id='2', last_update_time=1716986037626, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='file:///d:/rebeka/developer/courses/machine-learning-ops-zoomcamp/code/2_mlflow/mlruns/1', creation_time=1716946433085, experiment_id='1', last_update_time=1716946433085, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716946406946, experiment_id='0', last_update_time=1716946406946, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
runs = client.search_runs(
    experiment_ids="1",
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"],
)

for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 25329a592550460ab4004f056d268820, rmse: 6.3184
run id: cc215161cf47484896b5461629a1043a, rmse: 6.3355
run id: 50e10685ac60433db1bf5d27d95b47cd, rmse: 6.6188
run id: 8e107cf36a064d95bfe84f2472f394bf, rmse: 6.7423
run id: 9ecda438603145a786a05122669c4a76, rmse: 6.9077


In [6]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

run_id = "f9ecdaed61344d499361829bec8ae277"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716986128838, current_stage='None', description=None, last_updated_timestamp=1716986128838, name='nyc-taxi-regressor', run_id='f9ecdaed61344d499361829bec8ae277', run_link=None, source='file:///d:/rebeka/developer/courses/machine-learning-ops-zoomcamp/code/2_mlflow/mlruns/1/f9ecdaed61344d499361829bec8ae277/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [7]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Production
version: 2, stage: Staging
version: 3, stage: None


C:\Users\rebek\AppData\Local\Temp\ipykernel_12212\669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [8]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False,
)

C:\Users\rebek\AppData\Local\Temp\ipykernel_12212\2977303521.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716986128838, current_stage='Staging', description=None, last_updated_timestamp=1716986183599, name='nyc-taxi-regressor', run_id='f9ecdaed61344d499361829bec8ae277', run_link=None, source='file:///d:/rebeka/developer/courses/machine-learning-ops-zoomcamp/code/2_mlflow/mlruns/1/f9ecdaed61344d499361829bec8ae277/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [9]:
date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}",
)

<ModelVersion: aliases=[], creation_timestamp=1716986128838, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-05-29', last_updated_timestamp=1716986191120, name='nyc-taxi-regressor', run_id='f9ecdaed61344d499361829bec8ae277', run_link=None, source='file:///d:/rebeka/developer/courses/machine-learning-ops-zoomcamp/code/2_mlflow/mlruns/1/f9ecdaed61344d499361829bec8ae277/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [10]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)

    return df


def preprocess(df, dv):
    df["PU_DO"] = df["PULocationID"] + "_" + df["DOLocationID"]
    categorical = ["PU_DO"]
    numerical = ["trip_distance"]
    train_dicts = df[categorical + numerical].to_dict(orient="records")
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [11]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [12]:
client.download_artifacts(run_id=run_id, path="preprocessor", dst_path=".")

c:\Users\rebek\.virtualenvs\machine-learning-ops-zoomcamp-yuDfY6b0\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'd:\\rebeka\\developer\\courses\\machine-learning-ops-zoomcamp\\code\\2_mlflow\\preprocessor'

In [13]:
with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

X_test = preprocess(df, dv)

target = "duration"
y_test = df[target].values

In [14]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

c:\Users\rebek\.virtualenvs\machine-learning-ops-zoomcamp-yuDfY6b0\Lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


CPU times: total: 359 ms
Wall time: 2.77 s


{'rmse': 6.659623830022514}

In [15]:

%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

c:\Users\rebek\.virtualenvs\machine-learning-ops-zoomcamp-yuDfY6b0\Lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


CPU times: total: 57.7 s
Wall time: 1min 6s


{'rmse': 6.906080320724577}

In [16]:
client.transition_model_version_stage(
    name=model_name, version=3, stage="Production", archive_existing_versions=True
)

C:\Users\rebek\AppData\Local\Temp\ipykernel_12212\2772822162.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716986128838, current_stage='Production', description='The model version 3 was transitioned to Staging on 2024-05-29', last_updated_timestamp=1716986360098, name='nyc-taxi-regressor', run_id='f9ecdaed61344d499361829bec8ae277', run_link=None, source='file:///d:/rebeka/developer/courses/machine-learning-ops-zoomcamp/code/2_mlflow/mlruns/1/f9ecdaed61344d499361829bec8ae277/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>